In [1]:
import matplotlib.pyplot as plt
import gensim
import numpy as np

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
%matplotlib inline

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
text = open('eyecareFAQ.csv').read()

In [4]:
text

'ï»¿"I have ""spots"" floating around in my eye. Should I be worried?"\nI work on a computer all day. Can this hurt my eyes?\nI\'m interested in having LASIK done. What information do I need to know?\nCan you tell me what\'s wrong with my eyes?\nMy doctor refused to give me my contact lens prescription. Is this legal?\nAm I old enough to wear contact lenses?\n"What exactly does ""20/20 vision"" mean?"\nCan you help me find a doctor near me who...?\nI\'m looking for a specific type of contact lens. Do you know where I can find it?\nHow Often should I need to test my eyes?\nWhat are the symptoms of Cataract?\nDo I need surgery for Cataract? How would I know that?\nWhat is Myopia or Nearsightedness?\nWhat is Hyperopia or farsightedness?\nWhat is Presbyopia?\nWhat is the difference between Hyperopia and Presbyopia?\n"My eyes are fine, then why do I need to visit the doctor regularly?"\nWhat is Macular Degeneration?\nWho are at risk of developing Macular Degeneration?\nI have noticed small 

In [5]:
# my_stop_words = [u'say', u'\'s', u'Mr', u'be', u'said', u'says', u'saying']
# for stopword in my_stop_words:
#     lexeme = nlp.vocab[stopword]
#     lexeme.is_stop = True

In [6]:
doc = nlp((text))

In [7]:
doc

ï»¿"I have ""spots"" floating around in my eye. Should I be worried?"
I work on a computer all day. Can this hurt my eyes?
I'm interested in having LASIK done. What information do I need to know?
Can you tell me what's wrong with my eyes?
My doctor refused to give me my contact lens prescription. Is this legal?
Am I old enough to wear contact lenses?
"What exactly does ""20/20 vision"" mean?"
Can you help me find a doctor near me who...?
I'm looking for a specific type of contact lens. Do you know where I can find it?
How Often should I need to test my eyes?
What are the symptoms of Cataract?
Do I need surgery for Cataract? How would I know that?
What is Myopia or Nearsightedness?
What is Hyperopia or farsightedness?
What is Presbyopia?
What is the difference between Hyperopia and Presbyopia?
"My eyes are fine, then why do I need to visit the doctor regularly?"
What is Macular Degeneration?
Who are at risk of developing Macular Degeneration?
I have noticed small spots floating in front

In [8]:
# we add some words to the stop word list
texts, article, skl_texts = [], [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        # we add the lematized version of the word
        article.append(w.lemma_)
        print(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '\n':
        skl_texts.append(' '.join(article))
        texts.append(article)
        article = []

ï»¿"I
['ï»¿"I']
spot
['ï»¿"I', 'spot']
float
['ï»¿"I', 'spot', 'float']
eye
['ï»¿"I', 'spot', 'float', 'eye']
worried
['ï»¿"I', 'spot', 'float', 'eye', 'worried']
work
['work']
computer
['work', 'computer']
day
['work', 'computer', 'day']
hurt
['work', 'computer', 'day', 'hurt']
eye
['work', 'computer', 'day', 'hurt', 'eye']
interested
['interested']
have
['interested', 'have']
LASIK
['interested', 'have', 'LASIK']
information
['interested', 'have', 'LASIK', 'information']
need
['interested', 'have', 'LASIK', 'information', 'need']
know
['interested', 'have', 'LASIK', 'information', 'need', 'know']
tell
['tell']
wrong
['tell', 'wrong']
eye
['tell', 'wrong', 'eye']
doctor
['doctor']
refuse
['doctor', 'refuse']
contact
['doctor', 'refuse', 'contact']
lens
['doctor', 'refuse', 'contact', 'lens']
prescription
['doctor', 'refuse', 'contact', 'lens', 'prescription']
legal
['doctor', 'refuse', 'contact', 'lens', 'prescription', 'legal']
old
['old']
wear
['old', 'wear']
contact
['old', 'wear',

['potential', 'effect', 'limbal', 'relax']
incision
['potential', 'effect', 'limbal', 'relax', 'incision']
LRI
['potential', 'effect', 'limbal', 'relax', 'incision', 'LRI']
eye
['potential', 'effect', 'limbal', 'relax', 'incision', 'LRI', 'eye']
surgery
['potential', 'effect', 'limbal', 'relax', 'incision', 'LRI', 'eye', 'surgery']
prepare
['prepare']
limbal
['prepare', 'limbal']
relax
['prepare', 'limbal', 'relax']
incision
['prepare', 'limbal', 'relax', 'incision']
LRI
['prepare', 'limbal', 'relax', 'incision', 'LRI']
eye
['prepare', 'limbal', 'relax', 'incision', 'LRI', 'eye']
surgery
['prepare', 'limbal', 'relax', 'incision', 'LRI', 'eye', 'surgery']
LASIK
['LASIK']
surgery
['LASIK', 'surgery']
cause
['LASIK', 'surgery', 'cause']
dry
['LASIK', 'surgery', 'cause', 'dry']
eye
['LASIK', 'surgery', 'cause', 'dry', 'eye']
minimally
['minimally']
invasive
['minimally', 'invasive']
glaucoma
['minimally', 'invasive', 'glaucoma']
surgery
['minimally', 'invasive', 'glaucoma', 'surgery']
help

['treat']
retinal
['treat', 'retinal']
detachment
['treat', 'retinal', 'detachment']
happen
['happen']
eye
['happen', 'eye']
exam
['happen', 'eye', 'exam']
surgery
['surgery']
farsightedness
['surgery', 'farsightedness']
sight
['sight']
glaucoma
['sight', 'glaucoma']
long
['long']
eye
['long', 'eye']
appointment
['long', 'eye', 'appointment']
ophthalmoscope
['ophthalmoscope']
treat
['treat']
farsightedness
['treat', 'farsightedness']
long
['long']
stye
['long', 'stye']
risk
['risk']
factor
['risk', 'factor']
glaucoma
['risk', 'factor', 'glaucoma']
test
['test']
glaucoma
['test', 'glaucoma']
internal
['internal']
stye
['internal', 'stye']
symptom
['symptom']
secondary
['symptom', 'secondary']
glaucoma
['symptom', 'secondary', 'glaucoma']
get
['get']
stye
['get', 'stye']
long
['long']
glaucoma
['long', 'glaucoma']
treatment
['long', 'glaucoma', 'treatment']
prevent
['prevent']
vision
['prevent', 'vision']
problem
['prevent', 'vision', 'problem']
stye
['stye']
keep
['stye', 'keep']
come
[

['eye']
freckle
['eye', 'freckle']
form
['eye', 'freckle', 'form']
happen
['happen']
cataract
['happen', 'cataract']
interfere
['happen', 'cataract', 'interfere']
day
['happen', 'cataract', 'interfere', 'day']
day
['happen', 'cataract', 'interfere', 'day', 'day']
life
['happen', 'cataract', 'interfere', 'day', 'day', 'life']
internal
['internal']
limit
['internal', 'limit']
membrane
['internal', 'limit', 'membrane']
ILM
['internal', 'limit', 'membrane', 'ILM']
help
['internal', 'limit', 'membrane', 'ILM', 'help']
vitreomacular
['internal', 'limit', 'membrane', 'ILM', 'help', 'vitreomacular']
adhesion
['internal', 'limit', 'membrane', 'ILM', 'help', 'vitreomacular', 'adhesion']
VMA
['internal', 'limit', 'membrane', 'ILM', 'help', 'vitreomacular', 'adhesion', 'VMA']
mean
['mean']
hold
['mean', 'hold']
book
['mean', 'hold', 'book']
newspaper
['mean', 'hold', 'book', 'newspaper']
farther
['mean', 'hold', 'book', 'newspaper', 'farther']
away
['mean', 'hold', 'book', 'newspaper', 'farther', 

spread
['prevent', 'spread']
pinkeye
['prevent', 'spread', 'pinkeye']
medical
['medical']
issue
['medical', 'issue']
cause
['medical', 'issue', 'cause']
uveitis
['medical', 'issue', 'cause', 'uveitis']
corneal
['corneal']
abrasion
['corneal', 'abrasion']
eye
['eye']
heal
['eye', 'heal']
corneal
['eye', 'heal', 'corneal']
abrasion
['eye', 'heal', 'corneal', 'abrasion']
eye
['eye']
floater
['eye', 'floater']
normal
['eye', 'floater', 'normal']
treatment
['treatment']
cross
['treatment', 'cross']
eye
['treatment', 'cross', 'eye']
nystagmus
['treatment', 'cross', 'eye', 'nystagmus']
outlook
['outlook']
subconjunctival
['outlook', 'subconjunctival']
hemorrhage
['outlook', 'subconjunctival', 'hemorrhage']
excess
['excess']
tear
['excess', 'tear']
problem
['excess', 'tear', 'problem']
cataract
['cataract']
develop
['cataract', 'develop']
keratoconus
['keratoconus']
change
['keratoconus', 'change']
way
['keratoconus', 'change', 'way']
corneal
['corneal']
abrasion
['corneal', 'abrasion']
treat


...

['allergy', 'irritate', 'eye']
eye
['eye']
muscle
['eye', 'muscle']
surgery
['eye', 'muscle', 'surgery']
help
['eye', 'muscle', 'surgery', 'help']
nystagmus
['eye', 'muscle', 'surgery', 'help', 'nystagmus']
useless
['useless']
eye
['useless', 'eye']
exercise
['useless', 'eye', 'exercise']
glass
['glass']
contact
['glass', 'contact']
weaken
['glass', 'contact', 'weaken']
eyesight
['glass', 'contact', 'weaken', 'eyesight']
woman
['woman']
dry
['woman', 'dry']
eye
['woman', 'dry', 'eye']
vitamin
['woman', 'dry', 'eye', 'vitamin']
c
['woman', 'dry', 'eye', 'vitamin', 'c']
day
['woman', 'dry', 'eye', 'vitamin', 'c', 'day']
complication
['complication']
associate
['complication', 'associate']
retinopathy
['complication', 'associate', 'retinopathy']
prematurity
['complication', 'associate', 'retinopathy', 'prematurity']
ROP
['complication', 'associate', 'retinopathy', 'prematurity', 'ROP']
food
['food']
rich
['food', 'rich']
lutein
['food', 'rich', 'lutein']
zeaxanthin
['food', 'rich', 'lutei

myopia
['degenerative', 'myopia']
optic
['optic']
neuritis
['optic', 'neuritis']
treat
['optic', 'neuritis', 'treat']
risk
['risk']
factor
['risk', 'factor']
glaucoma
['risk', 'factor', 'glaucoma']
doctor
['doctor']
test
['doctor', 'test']
glaucoma
['doctor', 'test', 'glaucoma']
eye
['doctor', 'test', 'glaucoma', 'eye']
exam
['doctor', 'test', 'glaucoma', 'eye', 'exam']
guideline
['guideline']
prevent
['guideline', 'prevent']
eye
['guideline', 'prevent', 'eye']
infection
['guideline', 'prevent', 'eye', 'infection']
stop
['stop']
wear
['stop', 'wear']
contact
['stop', 'wear', 'contact']
lense
['stop', 'wear', 'contact', 'lense']
non
['non']
contact
['non', 'contact']
tonometry
['non', 'contact', 'tonometry']
treat
['treat']
irritated
['treat', 'irritated']
eye
['treat', 'irritated', 'eye']
cause
['treat', 'irritated', 'eye', 'cause']
extend
['treat', 'irritated', 'eye', 'cause', 'extend']
wear
['treat', 'irritated', 'eye', 'cause', 'extend', 'wear']
contact
['treat', 'irritated', 'eye',

lead
['cause', 'brain', 'lead']
double
['cause', 'brain', 'lead', 'double']
vision
['cause', 'brain', 'lead', 'double', 'vision']
seek
['seek']
immediate
['seek', 'immediate']
medical
['seek', 'immediate', 'medical']
care
['seek', 'immediate', 'medical', 'care']
think
['seek', 'immediate', 'medical', 'care', 'think']
glaucoma
['seek', 'immediate', 'medical', 'care', 'think', 'glaucoma']
iritis
['iritis']
treat
['iritis', 'treat']
home
['home']
follow
['home', 'follow']
care
['home', 'follow', 'care']
hyphema
['home', 'follow', 'care', 'hyphema']
symptom
['symptom']
hyphema
['symptom', 'hyphema']
eye
['eye']
doctor
['eye', 'doctor']
remove
['remove']
spinal
['remove', 'spinal']
fluid
['remove', 'spinal', 'fluid']
help
['remove', 'spinal', 'fluid', 'help']
papilledema
['remove', 'spinal', 'fluid', 'help', 'papilledema']
reason
['reason']
eye
['reason', 'eye']
doctor
['reason', 'eye', 'doctor']
eye
['reason', 'eye', 'doctor', 'eye']
freckle
['reason', 'eye', 'doctor', 'eye', 'freckle']
sy

progressive
['progressive']
lense
['progressive', 'lense']
look
['progressive', 'lense', 'look']
like
['progressive', 'lense', 'look', 'like']
lose
['lose']
weight
['lose', 'weight']
help
['lose', 'weight', 'help']
papilledema
['lose', 'weight', 'help', 'papilledema']
test
['test']
diagnose
['test', 'diagnose']
retinal
['test', 'diagnose', 'retinal']
vein
['test', 'diagnose', 'retinal', 'vein']
occlusion
['test', 'diagnose', 'retinal', 'vein', 'occlusion']
pinkeye
['pinkeye']
conjunctivitis
['pinkeye', 'conjunctivitis']
focal
['focal']
laser
['focal', 'laser']
therapy
['focal', 'laser', 'therapy']
retinal
['focal', 'laser', 'therapy', 'retinal']
vein
['focal', 'laser', 'therapy', 'retinal', 'vein']
occlusion
['focal', 'laser', 'therapy', 'retinal', 'vein', 'occlusion']
exudative
['exudative']
retinal
['exudative', 'retinal']
detachment
['exudative', 'retinal', 'detachment']
medication
['medication']
cause
['medication', 'cause']
papilledema
['medication', 'cause', 'papilledema']
sympto

['medicine']
help
['medicine', 'help']
treat
['medicine', 'help', 'treat']
open
['medicine', 'help', 'treat', 'open']
angle
['medicine', 'help', 'treat', 'open', 'angle']
glaucoma
['medicine', 'help', 'treat', 'open', 'angle', 'glaucoma']
condition
['condition']
raise
['condition', 'raise']
odd
['condition', 'raise', 'odd']
get
['condition', 'raise', 'odd', 'get']
open
['condition', 'raise', 'odd', 'get', 'open']
angle
['condition', 'raise', 'odd', 'get', 'open', 'angle']
glaucoma
['condition', 'raise', 'odd', 'get', 'open', 'angle', 'glaucoma']
symptom
['symptom']
open
['symptom', 'open']
angle
['symptom', 'open', 'angle']
glaucoma
['symptom', 'open', 'angle', 'glaucoma']
open
['open']
angle
['open', 'angle']
glaucoma
['open', 'angle', 'glaucoma']
eye
['open', 'angle', 'glaucoma', 'eye']
acne
['acne']
medicine
['acne', 'medicine']
cause
['acne', 'medicine', 'cause']
dry
['acne', 'medicine', 'cause', 'dry']
eye
['acne', 'medicine', 'cause', 'dry', 'eye']
pain
['pain']
reliever
['pain',

In [9]:
texts

[['ï»¿"I', 'spot', 'float', 'eye', 'worried'],
 ['work', 'computer', 'day', 'hurt', 'eye'],
 ['interested', 'have', 'LASIK', 'information', 'need', 'know'],
 ['tell', 'wrong', 'eye'],
 ['doctor', 'refuse', 'contact', 'lens', 'prescription', 'legal'],
 ['old', 'wear', 'contact', 'lense'],
 ['exactly', 'vision', 'mean'],
 ['help', 'find', 'doctor', 'near'],
 ['look', 'specific', 'type', 'contact', 'lens', 'know', 'find'],
 ['need', 'test', 'eye'],
 ['symptom', 'Cataract'],
 ['need', 'surgery', 'cataract', 'know'],
 ['myopia', 'nearsightedness'],
 ['hyperopia', 'farsightedness'],
 ['Presbyopia'],
 ['difference', 'Hyperopia', 'Presbyopia'],
 ['eye', 'fine', 'need', 'visit', 'doctor', 'regularly'],
 ['Macular', 'Degeneration'],
 ['risk', 'develop', 'Macular', 'Degeneration'],
 ['notice', 'small', 'spot', 'float', 'eye', 'cause', 'concern'],
 ['cause', 'dry', 'eye'],
 ['treat', 'dry', 'eye'],
 ['eyelid', 'start', 'twitch'],
 ['stop'],
 ['difference', 'ophthalmologist', 'optometrist', 'optici

In [10]:
len(texts)

3062

In [11]:
skl_texts

['ï»¿"I spot float eye worried',
 'work computer day hurt eye',
 'interested have LASIK information need know',
 'tell wrong eye',
 'doctor refuse contact lens prescription legal',
 'old wear contact lense',
 'exactly vision mean',
 'help find doctor near',
 'look specific type contact lens know find',
 'need test eye',
 'symptom Cataract',
 'need surgery cataract know',
 'myopia nearsightedness',
 'hyperopia farsightedness',
 'Presbyopia',
 'difference Hyperopia Presbyopia',
 'eye fine need visit doctor regularly',
 'Macular Degeneration',
 'risk develop Macular Degeneration',
 'notice small spot float eye cause concern',
 'cause dry eye',
 'treat dry eye',
 'eyelid start twitch',
 'stop',
 'difference ophthalmologist optometrist optician',
 'normal vision develop',
 'eye work eye anatomy',
 'child eye examine',
 'adult eye examine',
 'appointment routine eye examination',
 'pink eye contagious',
 'medical term nearsightedness farsightedness',
 'visual acuity',
 'legal blindness',
 'v

In [12]:
bigram = gensim.models.Phrases(texts)

In [13]:
texts = [bigram[line] for line in texts]

In [14]:
texts[1][0:10]

['work', 'computer', 'day', 'hurt', 'eye']

In [15]:
texts[5]

['old', 'wear_contact', 'lense']

In [16]:
texts

[['ï»¿"I', 'spot', 'float', 'eye', 'worried'],
 ['work', 'computer', 'day', 'hurt', 'eye'],
 ['interested', 'have', 'LASIK', 'information', 'need', 'know'],
 ['tell', 'wrong', 'eye'],
 ['doctor', 'refuse', 'contact', 'lens', 'prescription', 'legal'],
 ['old', 'wear_contact', 'lense'],
 ['exactly', 'vision', 'mean'],
 ['help', 'find', 'doctor', 'near'],
 ['look', 'specific', 'type', 'contact', 'lens', 'know', 'find'],
 ['need', 'test', 'eye'],
 ['symptom', 'Cataract'],
 ['need', 'surgery', 'cataract', 'know'],
 ['myopia', 'nearsightedness'],
 ['hyperopia', 'farsightedness'],
 ['Presbyopia'],
 ['difference', 'Hyperopia', 'Presbyopia'],
 ['eye', 'fine', 'need', 'visit', 'doctor', 'regularly'],
 ['Macular', 'Degeneration'],
 ['risk', 'develop', 'Macular', 'Degeneration'],
 ['notice', 'small', 'spot', 'float', 'eye', 'cause', 'concern'],
 ['cause', 'dry', 'eye'],
 ['treat', 'dry', 'eye'],
 ['eyelid', 'start', 'twitch'],
 ['stop'],
 ['difference', 'ophthalmologist', 'optometrist', 'optician'

In [17]:
bigram = gensim.models.Phrases(texts)

In [18]:
texts = [bigram[line] for line in texts]

In [19]:
texts[5]

['old', 'wear_contact_lense']

In [20]:
texts

[['ï»¿"I', 'spot', 'float', 'eye', 'worried'],
 ['work', 'computer', 'day', 'hurt', 'eye'],
 ['interested', 'have', 'LASIK', 'information', 'need', 'know'],
 ['tell', 'wrong', 'eye'],
 ['doctor', 'refuse', 'contact', 'lens', 'prescription', 'legal'],
 ['old', 'wear_contact_lense'],
 ['exactly', 'vision', 'mean'],
 ['help', 'find', 'doctor', 'near'],
 ['look', 'specific', 'type', 'contact', 'lens', 'know', 'find'],
 ['need', 'test', 'eye'],
 ['symptom', 'Cataract'],
 ['need', 'surgery', 'cataract', 'know'],
 ['myopia', 'nearsightedness'],
 ['hyperopia', 'farsightedness'],
 ['Presbyopia'],
 ['difference', 'Hyperopia', 'Presbyopia'],
 ['eye', 'fine', 'need', 'visit', 'doctor', 'regularly'],
 ['Macular', 'Degeneration'],
 ['risk', 'develop', 'Macular', 'Degeneration'],
 ['notice', 'small', 'spot', 'float', 'eye', 'cause', 'concern'],
 ['cause', 'dry', 'eye'],
 ['treat', 'dry', 'eye'],
 ['eyelid', 'start', 'twitch'],
 ['stop'],
 ['difference', 'ophthalmologist', 'optometrist', 'optician'],


In [21]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [22]:
corpus[1][0:10]

[(0, 1), (5, 1), (6, 1), (7, 1), (8, 1)]

In [23]:
# lsimodel = LsiModel(corpus=corpus, num_topics=3, id2word=dictionary)

In [24]:
# lsimodel.show_topics(num_topics=5)  # Showing only the top 5 topics

In [25]:
# hdpmodel = HdpModel(corpus=corpus, T=3, id2word=dictionary)

In [26]:
# hdpmodel.show_topics()

In [27]:
ldamodel = LdaModel(corpus=corpus, num_topics=3, id2word=dictionary)

In [28]:
ldamodel.show_topics()

[(0,
  '0.097*"eye" + 0.036*"treat" + 0.026*"help" + 0.021*"vision" + 0.021*"doctor" + 0.020*"uveitis" + 0.015*"drop" + 0.015*"cataract" + 0.014*"happen" + 0.014*"retinal_vein_occlusion"'),
 (1,
  '0.122*"eye" + 0.041*"symptom" + 0.037*"surgery" + 0.034*"glaucoma" + 0.024*"exam" + 0.023*"pinkeye" + 0.022*"doctor" + 0.021*"need" + 0.014*"prevent" + 0.014*"affect"'),
 (2,
  '0.103*"eye" + 0.051*"cause" + 0.049*"treat" + 0.023*"vision" + 0.019*"problem" + 0.016*"dry" + 0.014*"diagnose" + 0.011*"happen" + 0.011*"infection" + 0.010*"cornea"')]

In [35]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.103149 -0.088753       1        1  36.514641
1     -0.140519 -0.032819       2        1  35.281292
0      0.037370  0.121572       3        1  28.204071, topic_info=            Term        Freq       Total Category  logprob  loglift
58         treat  287.000000  287.000000  Default  30.0000  30.0000
134     glaucoma  135.000000  135.000000  Default  29.0000  29.0000
53         cause  239.000000  239.000000  Default  28.0000  28.0000
365      uveitis   60.000000   60.000000  Default  27.0000  27.0000
132         exam   93.000000   93.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
253     diagnose   19.922817   70.099487   Topic3  -4.9202   0.0077
403      pinkeye   22.319277  120.344452   Topic3  -4.8067  -0.4192
57           dry   20.305647  114.388855   Topic3  -4.9012  -0.4630
330  papilledema   16.589647   42.038559   Topic3  -5.1033   0.3359
68         child   15.259373   58.101002   Topic3  -5.1869  -0.0713

[221 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
547       2  0.940700  Fuchs_dystrophy
9         1  0.168726            LASIK
9         2  0.795422            LASIK
9         3  0.024104            LASIK
234       1  0.038273           affect
...     ...       ...              ...
8         1  0.050773             work
8         2  0.812363             work
8         3  0.152318             work
353       1  0.310183      xanthelasma
353       2  0.664678      xanthelasma

[329 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [30]:
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(hdpmodel, corpus, dictionary)

In [31]:
# lsitopics = [[word for word, prob in topic] for topicid, topic in lsimodel.show_topics(formatted=False)]

# hdptopics = [[word for word, prob in topic] for topicid, topic in hdpmodel.show_topics(formatted=False)]

# ldatopics = [[word for word, prob in topic] for topicid, topic in ldamodel.show_topics(formatted=False)]

In [32]:
# lsi_coherence = CoherenceModel(topics=lsitopics[:10], texts=texts, dictionary=dictionary, window_size=10).get_coherence()

# hdp_coherence = CoherenceModel(topics=hdptopics[:10], texts=texts, dictionary=dictionary, window_size=10).get_coherence()

# lda_coherence = CoherenceModel(topics=ldatopics, texts=texts, dictionary=dictionary, window_size=10).get_coherence()

In [33]:
# def evaluate_bar_graph(coherences, indices):
#     """
#     Function to plot bar graph.
    
#     coherences: list of coherence values
#     indices: Indices to be used to mark bars. Length of this and coherences should be equal.
#     """
#     assert len(coherences) == len(indices)
#     n = len(coherences)
#     x = np.arange(n)
#     plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
#     plt.xlabel('Models')
#     plt.ylabel('Coherence Value')

In [34]:
# evaluate_bar_graph([hdp_coherence, lda_coherence],
#                    ['HDP', 'LDA'])